# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# load data from database
engine = create_engine('sqlite:///ProjectData.db')
df = pd.read_sql_table(table_name = 'ProjectData', con = engine)
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    stemmed = [PorterStemmer().stem(w) for w in words]
    lemmatized = [WordNetLemmatizer().lemmatize(w) for w in stemmed]
    return lemmatized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 24)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred = pipeline.predict(X_test)

In [7]:
for i in range(y_pred.shape[1]):
    print('------------------------------------------')
    print('   ' + y_test.columns[i])
    print('------------------------------------------')
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))
    print('')

------------------------------------------
   related
------------------------------------------
             precision    recall  f1-score   support

          0       0.66      0.46      0.55      1540
          1       0.84      0.92      0.88      4958
          2       0.32      0.34      0.33        56

avg / total       0.80      0.81      0.80      6554


------------------------------------------
   request
------------------------------------------
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5410
          1       0.81      0.45      0.58      1144

avg / total       0.88      0.89      0.87      6554


------------------------------------------
   offer
------------------------------------------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6534
          1       0.00      0.00      0.00        20

avg / total       0.99      1.00      1.00      6554


----------

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
accuracy = (y_pred == y_test.values).mean()
print('Accuracy {:.3f}'.format(accuracy))

Accuracy 0.945


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
    'clf__estimator__n_estimators': [3, 5],
    'clf__estimator__min_samples_split': [2,4,5],
    'vect__max_df': [2, 3, 4]
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

model = cv
model.fit(X_train, y_train)

accuracy = (y_pred == y_test.values).mean()
print('Accuracy {:.3f}'.format(accuracy))

In [ ]:
ph

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [10]:
y_pred = pipeline.predict(X_test)
for i in range(y_pred.shape[1]):
    print('------------------------------------------')
    print('   ' + y_test.columns[i])
    print('------------------------------------------')
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))
    print('')

------------------------------------------
   related
------------------------------------------
             precision    recall  f1-score   support

          0       0.66      0.46      0.55      1540
          1       0.84      0.92      0.88      4958
          2       0.32      0.34      0.33        56

avg / total       0.80      0.81      0.80      6554


------------------------------------------
   request
------------------------------------------
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5410
          1       0.81      0.45      0.58      1144

avg / total       0.88      0.89      0.87      6554


------------------------------------------
   offer
------------------------------------------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6534
          1       0.00      0.00      0.00        20

avg / total       0.99      1.00      1.00      6554


----------

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
accuracy = (y_pred == y_test.values).mean()
print('Accuracy {:.3f}'.format(accuracy))

Accuracy 0.945


### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.